# Lab | Web Scraping

Welcome to the "Books to Scrape" Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the data available on Books to Scrape, an online platform showcasing a wide variety of books. As data analyst, you have been tasked with scraping a specific subset of book data from Books to Scrape to assist publishing companies in understanding the landscape of highly-rated books across different genres. Your insights will help shape future book marketing strategies and publishing decisions.

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The publishing industry, much like others, utilizes data analytics to understand market trends, reader preferences, and the performance of books based on factors such as genre, author, and ratings. Books to Scrape serves as a rich source of such data, offering detailed information about a diverse range of books, making it an ideal platform for extracting insights to aid in informed decision-making within the literary world.

**Task**

Your task is to create a Python script using BeautifulSoup and pandas to scrape Books to Scrape book data, focusing on book ratings and genres. The script should be able to filter books with ratings above a certain threshold and in specific genres. Additionally, the script should structure the scraped data in a tabular format using pandas for further analysis.

**Expected Outcome**

A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`. The function should scrape book data from the "Books to Scrape" website and return a `pandas` DataFrame with the following columns:

**Expected Outcome**

- A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`.
- The function should return a DataFrame with the following columns:
  - **UPC**: The Universal Product Code (UPC) of the book.
  - **Title**: The title of the book.
  - **Price (£)**: The price of the book in pounds.
  - **Rating**: The rating of the book (1-5 stars).
  - **Genre**: The genre of the book.
  - **Availability**: Whether the book is in stock or not.
  - **Description**: A brief description or product description of the book (if available).
  
You will execute this script to scrape data for books with a minimum rating of `4.0 and above` and a maximum price of `£20`. 

Remember to experiment with different ratings and prices to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [Books to Scrape](https://books.toscrape.com/)


**Hint**

Your first mission is to familiarize yourself with the **Books to Scrape** website. Navigate to [Books to Scrape](http://books.toscrape.com/) and explore the available books to understand their layout and structure. 

Next, think about how you can set parameters for your data extraction:

- **Minimum Rating**: Focus on books with a rating of 4.0 and above.
- **Maximum Price**: Filter for books priced up to £20.

After reviewing the site, you can construct a plan for scraping relevant data. Pay attention to the details displayed for each book, including the title, price, rating, and availability. This will help you identify the correct HTML elements to target with your scraping script.

Make sure to build your scraping URL and logic based on the patterns you observe in the HTML structure of the book listings!


---

**Best of luck! Immerse yourself in the world of books, and may the data be with you!**

**Important Note**:

In the fast-changing online world, websites often update and change their structures. When you try this lab, the **Books to Scrape** website might differ from what you expect.

If you encounter issues due to these changes, like new rules or obstacles preventing data extraction, don’t worry! Get creative.

You can choose another website that interests you and is suitable for scraping data. Options like Wikipedia, The New York Times, or even library databases are great alternatives. The main goal remains the same: extract useful data and enhance your web scraping skills while exploring a source of information you enjoy. This is your opportunity to practice and adapt to different web environments!

In [100]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import re


base_url = 'https://books.toscrape.com/catalogue/'


def get_soup(url):
    data = requests.get(url)
    soup = bs(data.content, 'html.parser')
    return soup


def scrape_books(min_rating=4, max_price=20):
    all_books = []

    next_page = 1
    last_page = 0  # 0 for non stop until error

    if last_page == 0:
        last_page = float('inf')

    while last_page >= next_page:
        url = f'https://books.toscrape.com/catalogue/page-{next_page}.html'
        print(f'Scraping {url}')

        try:
            all_books.extend(scrape_page(
                url, rating=min_rating, price=max_price))
        except Exception as e:
            print(e)
            break

        next_page += 1

    print('all books :', all_books)
    return all_books


def filter_book(book_data: dict, rating, price):
    if book_data['rating'] >= rating and book_data['price'] <= price:
        return True
    return False


def scrape_page(url, **kwargs):
    soup = get_soup(url)
    books = soup.find_all('article', class_='product_pod')

    page_books = []

    for book in books:
        book_url = base_url + book.h3.a['href']
        print(book_url)
        book_data = get_book_data(book_url)
        print(book_data)

        if filter_book(book_data, **kwargs):
            page_books.append(book_data)
        else:
            print(f'Book was filtered out')
        print('\n__________\n')
    print('page books :', page_books)
    return page_books


def get_book_data(url):
    soup = get_soup(url)

    book_data = {
        "upc": get_upc(soup),
        "title": get_title(soup),
        "price": get_price(soup),
        "rating": get_rating(soup),
        "genre": get_genre(soup),
        "availabilty": get_availability(soup),
        "description": get_description(soup),
    }

    print('book data :', book_data)
    return book_data


def get_upc(soup):
    upc = get_value_from_label(soup, 'UPC')
    print('upc :', upc)
    return upc


def get_title(soup):
    title = soup.find('h1').text
    print('title :', title)
    return title


def get_price(soup):
    price = get_value_from_label(soup, 'Price (incl. tax)')
    price_numerical = float(price[1:])

    print('price :', price_numerical)
    return price_numerical


def get_rating(soup):
    rating_text = soup.find('p', class_='star-rating')['class'][1]
    rating_dict = {'One': 1, 'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5}
    rating_score = rating_dict[rating_text]

    print('rating :', rating_score)
    return rating_score


def get_genre(soup):
    genre = soup.find('ul', class_='breadcrumb').find_all('a')[2].text
    print('genre :', genre)
    return genre


def get_availability(soup):
    availability = get_value_from_label(soup, 'Availability')
    stock = int(re.search(r'\d+', availability)[0])
    print(stock)
    print('availability :', stock)
    return stock


def get_description(soup):
    description = soup.find(
        'div', id='product_description').find_next('p').text
    print('desc :', description)
    return description


def get_value_from_label(soup, label):
    rows = soup.find('table', class_='table-striped').find_all('tr')
    for row in rows:
        if row.th.text == label:
            return row.td.text
    return None


def merge_data(*dicts):
    result = {}
    for dictionary in dicts:
        result.update(dictionary)
    return result


def export_books(df):
    df.to_csv('books.csv', index=False)
    print('books exported to csv')

In [101]:
df = pd.DataFrame(scrape_books(4, 20))


Scraping https://books.toscrape.com/catalogue/page-1.html
https://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html
upc : a897fe39b1053632
title : A Light in the Attic
price : 51.77
rating : 3
genre : Poetry
22
availability : 22
desc : It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love th It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love

In [102]:
display(df)
df.dtypes

,upc,title,price,rating,genre,availabilty,description
0,ce6396b0f23f6ecc,Set Me Free,17.46,5,Young Adult,19,Aaron Ledbetter’s future had been planned out ...


upc             object
title           object
price          float64
rating           int64
genre           object
availabilty      int64
description     object
dtype: object

In [103]:
export_books(df)

books exported to csv
